In [1]:
!pip install sqlalchemy

DEPRECATION: tensorflow-examples e3849ccb6d981aad311f0174281c8575d5e21646- has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of tensorflow-examples or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [4]:
import os
from sqlalchemy import create_engine
import pandas as pd


class CustomeDataSet():
    def __init__(self, traindata_url, tipo_dados='EndPlate', transform=None, size=(768, 768)):
        # Recuperando dados com dataframe --> Pelo visto é a melhor forma ...
        self.df = pd.read_sql(f"SELECT * FROM dados_table WHERE name='{tipo_dados}'", con=create_engine(f'sqlite:///{traindata_url}/database.db') )
        self.traindata_url = traindata_url # path da pasta com os dados
        self.nome_perfil = self.df['nome_perfil'] # Serie com os nome dos perfis
        self.solicitacao = self.df['solicitacao'] # Serie com as solicitacoes utilizadas
        self.img_uuid = self.df['uuid'] # Identificador unico da aplicação
        self.viga = self.df['nome_perfil']
        self.coluna = self.df['nome_perfil']
        self.fs = self.df['fs']
        self.qntd_parafusos = self.df['qntd_parafusos']
        self.carregamento = self.df['solicitacao']
        self.bitola = self.df['bitola_parafuso']
        self.size = size # Tamanho padrão da imagem
        self.transform = transform # Transformações a serem aplicadas nas imagens


    def __len__(self):
        return len(self.df)


    def __getitem__(self, index:int):

        # Recuperando uuid do elemento
        uuid_str = self.img_uuid[index]
        img_uuid = f"{uuid_str[:8]}-{uuid_str[8:12]}-{uuid_str[12:16]}-{uuid_str[16:20]}-{uuid_str[20:]}"

        # Escrevendo o propmt da imagem
        
        caption = f'Complete the masked region on the left with the cross-sectional view, and the masked region on the right with the plan detail of a steel structural connection between an I-beam and an I-column, designed for a vertical load of {self.carregamento[index]} kN.Use the following information: seção viga: {self.viga[index]}, Bitola do parafuso: {self.bitola[index]}, quantidade de parafusos: {self.qntd_parafusos[index]}, FS: {self.fs[index]}. Use technical representation according to structural drawing standards, including visible profiles, bolts, and connection plates.'


        return caption, img_uuid

In [6]:
dados = CustomeDataSet(r'C:\Users\Breno HM Rodrigues\Desktop\Data', tipo_dados='LLCP')
dados[0]

('Complete the masked region on the left with the cross-sectional view, and the masked region on the right with the plan detail of a steel structural connection between an I-beam and an I-column, designed for a vertical load of 50.0 kN.Use the following information: seção viga: W150x13, Bitola do parafuso: 16, quantidade de parafusos: 2, FS: 0.5086628152794952. Use technical representation according to structural drawing standards, including visible profiles, bolts, and connection plates.',
 '53b460cc-78a3-48ca-9023-4bd2f08fe7fa')

In [12]:
import random

random.seed(42)  # define a seed fixa

samples = 100

already_in = []

LLCP = True

# Escreve o arquivo com separador "#;#"
with open("data/train.csv", "w", encoding="utf-8") as f:
    for i in range(samples//3):
        if i>samples//3*0.5:
            dados = CustomeDataSet(r'C:\Users\Breno HM Rodrigues\Desktop\Data', tipo_dados='EndPlate')
        else:
            dados = CustomeDataSet(r'C:\Users\Breno HM Rodrigues\Desktop\Data', tipo_dados='LLCP')
            if LLCP:
                already_in = []
                LLCP = False
            
        # Garantindo que não siga nenhuma sequência
        new_date = random.randint(0,len(dados))
        print(new_date)
        while new_date in already_in:
           
            new_date = random.randint(0,len(dados))
            
        caption, img_uuid  = dados[new_date]
        already_in.append(new_date)
        
        for el in ['top','lateral','frontal']:
            img_target = os.path.join('data','data_train','img',el, f'{img_uuid}.png')
            img_mask = os.path.join('data','data_train','mask',el, f'{img_uuid}.png')
            img_base = os.path.join('data','data_train','base',el, f'{img_uuid}.png')
  
            
            linha = "#;#".join([
                img_base.replace('\\', '/'),
                img_mask.replace('\\', '/'),
                '',
                img_target.replace('\\', '/'),
                caption,
                'local_editing'
            ])
            f.write(linha + "\n")

print("Arquivo 'data/train.csv' salvo com sucesso!")

327
57
12
140
125
114
71
52
279
44
302
216
16
15
47
111
119
258
308
13
287
101
332
279
112
229
301
142
3
81
216
142
110
Arquivo 'data/train.csv' salvo com sucesso!
